In [155]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime
from pytz import timezone
import pandas_datareader as pdr
import json
from bson.json_util import dumps
import numpy as np
import datetime

In [159]:
# 현재 시간을 UTC로 표시한다
start_date = datetime.datetime.now() - datetime.timedelta(days=91)
start_timestamp = start_date.timestamp()


In [17]:
#몽고 db 연동
client = MongoClient("mongodb://localhost:27017/")
db = client.stock_study
company_info = db.company_info
company_financial_info = db.company_financial_info
company_stock = db.company_stock

In [18]:
#회사 정보, 실적정보 전부가져오기
company_info_df = company_info.find()
company_financial_info_df = company_financial_info.find()

In [19]:
#데이터프레임화
company_info_df = pd.DataFrame(json.loads(dumps(company_info_df)))
company_financial_info_df = pd.DataFrame(json.loads(dumps(company_financial_info_df)))

In [67]:
# \xa0값 들어온것 "N/A"처리 함수
def nullcheck(str_s):
    if str_s == "\xa0":
        return "N/A"
    else:
        return str_s

In [69]:
#각 회사코드별로 
company_cc = []
fr = []
sr = []
ty = []
for i in range(len(company_info_df)):
    company_code = company_info_df['회사코드'][i]
    finanancial_report = company_financial_info_df[company_financial_info_df['회사코드'] == company_code].reset_index()['data']
    if len(finanancial_report) == 0:
        company_cc.append(company_code)
        fr.append('N/A')
        sr.append('N/A')
        ty.append('N/A')
    else:
        company_cc.append(company_code)
        fr.append(nullcheck(finanancial_report[7]['영업이익(억원)']))
        sr.append(nullcheck(finanancial_report[8]['영업이익(억원)']))
        ty.append(nullcheck(finanancial_report[9]['영업이익(억원)']))
    print(company_code)

326030
353200
344820
308170
322000
272210
317400
013890
336370
336260
036420
003670
307950
192650
192080
316140
298690
267850
306200
293480
010400
300720
091810
286940
298040
298050
298020
298000
294870
018250
068270
284740
285130
282330
092780
272450
281820
282690
272550
055490
280360
271980
035720
271560
268280
267290
251270
267270
267260
267250
264900
145720
111110
143210
241560
207940
234080
241590
229640
249420
248170
016740
026960
123890
034830
195870
070960
101530
004440
194370
152330
226320
075580
214330
089590
227840
079550
039570
002690
095570
214320
214420
085620
214390
210980
138250
145210
213500
210540
028260
112610
018260
204320
200880
192400
133820
027410
192820
185750
006880
183190
064350
180640
181710
175330
155660
170900
013870
037560
163560
161890
161390
161000
068400
014710
144620
079980
007070
129260
145990
039130
078520
023000
031430
047810
071840
069640
138490
139480
139130
053210
019440
138040
023350
138930
020150
011210
136490
134380
134790
082740
117580
126560

200470
200670
208710
189860
067390
080580
208350
160550
124500
142280
187220
208370
208140
149980
084650
200710
200780
207760
196170
206400
178920
140520
173940
205470
200230
194510
204840
204620
196490
205500
205100
191420
192440
182690
143540
203650
041920
194480
203690
192410
198440
187270
177830
090410
004650
192390
059120
200130
105550
196450
071850
187790
192250
138080
053300
154040
090850
184230
049080
150840
085810
170030
067570
134580
182360
171120
171010
161570
138360
076610
168330
131970
150900
170920
119850
092040
130500
170790
151860
089600
950130
141080
158310
097800
104540
159580
114810
153490
099190
141020
159910
950110
046970
113810
141070
155650
149950
121850
151910
097520
153460
106520
137400
141000
072950
143240
126870
147830
091590
140410
104830
127120
130580
131090
100660
115480
139670
121800
138070
123570
122870
136510
089530
089030
101240
007820
115530
043290
090360
131220
131390
143160
137950
040910
123330
139050
089980
138610
109080
108380
019770
137940
131100

In [72]:
financial_rr = pd.DataFrame({'회사코드':company_cc,'first_revenue':fr,'second_revenue':sr,'third_revenue':ty})

In [84]:
company_df = pd.merge(company_info_df,financial_rr,on=['회사코드'])

In [106]:
company_stock_df = pd.DataFrame(json.loads(dumps(company_stock.find({'종목코드':'005930'}))))
stocks = company_stock_df['data']

In [ ]:
company_code = []
result = []
for stock_code in company_info_df['회사코드']:
    company_stock_df = pd.DataFrame(json.loads(dumps(company_stock.find({'종목코드':stock_code}))))
    Date = []
    Close = []
    if len(company_stock_df) < 30 :
        company_code.append(stock_code)
        result.append('N/A')
    else:
        stocks = company_stock_df['data']
        for i in range(len(stocks)):
            #3달전 timestamp와 비교해서 큰것만 가져옴
            if  stocks[i]['Date'] > start_timestamp:
                Date.append(stocks[i]['Date'])
                Close.append(stocks[i]['Close'])
        company_code.append(stock_code)
        result.append(np.corrcoef(Date,Close)[1][0])
    

In [134]:
financial_rr2 = pd.DataFrame({'회사코드':company_code,'result':result})

In [136]:
company_df = pd.merge(company_df,financial_rr2)

In [139]:
company_df.to_csv('company_df.csv',encoding='ms949')

1587275573.212735